In [1]:
import tensorflow as tf

### tf.data API -- for building efficient data pipelines

In [2]:
X = tf.range(10)

In [3]:
dataset = tf.data.Dataset.from_tensor_slices(X)

In [4]:
for data in dataset:
    print(data)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


In [5]:
dataset2 = tf.data.Dataset.range(10)

In [7]:
for i in dataset2:
    print(i)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)


### chaining transforms

In [9]:
dataset = dataset.repeat(3).batch(7)

In [10]:
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9], shape=(2,), dtype=int32)


In [11]:
dataset2 = dataset2.map(lambda x: x*2)

In [13]:
for j in dataset2:
    print(j)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(10, shape=(), dtype=int64)
tf.Tensor(12, shape=(), dtype=int64)
tf.Tensor(14, shape=(), dtype=int64)
tf.Tensor(16, shape=(), dtype=int64)
tf.Tensor(18, shape=(), dtype=int64)


### Example

In [14]:
mnist = tf.keras.datasets.mnist

In [15]:
(train_x, train_y), (test_x, test_y) = mnist.load_data()

In [33]:
train_x, test_x = tf.cast(train_x/255.0, tf.float32), tf.cast(test_x/255.0, tf.float32)

In [34]:
train_y, test_y = tf.cast(train_y, tf.int64), tf.cast(test_y, tf.int64)

In [35]:
batch_size = 32
buffer_size = 10000


In [39]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(32).shuffle(buffer_size)

In [40]:
train_dataset = train_dataset.map(lambda x, y: (tf.image.random_flip_left_right(x), y))

In [41]:
train_dataset = train_dataset.repeat()

In [42]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).batch(32).shuffle(buffer_size)

In [43]:
test_dataset = test_dataset.repeat()

### Model definition using Keras Sub classing

In [25]:
class Mymodel(tf.keras.Model):
    def __init__(self, num_classes=10):
        super(Mymodel, self).__init__()
        #define your layers here
        inputs = tf.keras.Input(shape=(28, 28))
        self.x0 = tf.keras.layers.Flatten()
        self.x1 = tf.keras.layers.Dense(512, activation='relu', name='d1')
        self.x2 = tf.keras.layers.Dropout(0.2)
        self.predictions = tf.keras.layers.Dense(10, activation='softmax', name='d2')
        
    
    def call(self, inputs):
        x = self.x0(inputs)
        x = self.x1(x)
        x = self.x2(x)
        return self.predictions(x)

### Using the model with data pipelines

In [26]:
model = Mymodel()

In [27]:
steps_per_epoch = len(train_x)//batch_size

In [28]:
optimizer = tf.keras.optimizers.Adam()

In [29]:
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [44]:
model.fit(train_dataset, epochs=10, steps_per_epoch=steps_per_epoch)

Train for 1875 steps
Epoch 1/10
1875/1875 [==============================] - 13s 7ms/step - loss: 1.1832 - accuracy: 0.6566
Epoch 2/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.6684 - accuracy: 0.7998
Epoch 3/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.5902 - accuracy: 0.8219
Epoch 4/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.5495 - accuracy: 0.8348
Epoch 5/10
1875/1875 [==============================] - 12s 7ms/step - loss: 0.5180 - accuracy: 0.8446
Epoch 6/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.4911 - accuracy: 0.8517
Epoch 7/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.4653 - accuracy: 0.8597
Epoch 8/10
1875/1875 [==============================] - 12s 7ms/step - loss: 0.4393 - accuracy: 0.8680
Epoch 9/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.4138 - accuracy: 0.8765
Epoch 10/10
1875/1875 [=============================

In [57]:
loss, accuracy = model.evaluate(test_x, test_y, verbose=0)
print(loss, accuracy)

0.3646162013292313 0.8884
